Rectangle frames

In [1]:
import cv2
import numpy as np
from joblib import load

# Load the SVM model from the pickle file
model = load('PlasticDetection.pkl')

# Define the boundaries of the plastic bottle (in RGB color space)
lower = np.array([100, 100, 100])
upper = np.array([255, 255, 255])

# Create a VideoCapture object to read from the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    
    # Convert the frame from BGR to RGB color space
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Apply the color threshold to extract the plastic bottle regions
    mask = cv2.inRange(frame_rgb, lower, upper)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Make predictions for each detected region
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (100, 100))
        roi_flattened = roi_resized.reshape(1, -1).astype('float32') / 255
        prediction = model.predict(roi_flattened)
        
        # Draw a rectangle around the detected plastic bottle and label it as "Plastic" or "Non-Plastic"
        if prediction == 'Plastic':
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'Plastic', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    
    # Display the frame with the detected regions and predictions
    cv2.imshow('Plastic Detection', frame)
    
    # Exit the program if the user presses the 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

# Release the VideoCapture object and close all windows
cap.release()
cv2.destroyAllWindows()


Single Snap

In [11]:
import cv2
import joblib

# Load the saved SVM model
model = joblib.load('PlasticDetection.pkl')

# Set the webcam source
cap = cv2.VideoCapture(0)

# Capture an image from the webcam
ret, frame = cap.read()

# Resize the image to 100x100
resized_frame = cv2.resize(frame, (100, 100))

# Flatten the image and normalize the pixel values
flattened_frame = resized_frame.reshape(1, -1).astype('float32') / 255

# Pass the image through the model for prediction
prediction = model.predict(flattened_frame)

# Draw a bounding box around the detected plastic bottle
if prediction == 'plastic':
    cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), thickness=2)

# Display the image and the predicted label
cv2.imwrite('test.jpg',frame)
image = cv2.imread("test.jpg")
cv2.imshow(str(prediction), image)
print('Prediction:', prediction)
cv2.waitKey(0)

# Release the webcam source and close the window
cap.release()
cv2.destroyAllWindows()


Prediction: ['Non Plastic']


In [12]:
import cv2

# Set the webcam source
cap = cv2.VideoCapture(1)
model = joblib.load('PlasticDetection.pkl')

# Capture an image from the webcam
ret, frame = cap.read()

# Release the webcam source
cap.release()

# Save the captured image to a file
cv2.imwrite('captured_image.jpg', frame)

# Display the captured image
cv2.imshow('image', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
